# Homework 3

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2024

In [28]:
import numpy as np
import pandas as pd

import platform
if platform.system() == "Windows":
    BASE_DIR = r'C:\Users\Alex\Desktop\Academic\UChicago\FINM 37500\finm-fiderivs-2024'
else:
    BASE_DIR = r'/Users/alexhuang/Documents/Academic/FINM 37500/finm-fiderivs-2024'


import sys, os
sys.path.append(BASE_DIR)

from cmds.binomial import *
from cmds.ratecurves import *
from cmds.volskew import *
from cmds.ficcvol import *
from cmds.fi_binomial_model import FIBinomialModel

# 1. Modeling the Volatility Smile

## Swaption Vol Data

The file `data/swaption_vol_data_2024-02-20.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2024-02-20`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_quotes_2024-02-20.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.
* This cap data would be helpful in calibrating a binomial tree, but this problem focuses on Black's formula and SABR.

## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

In [29]:

class FIBinomialModel():
    def __init__(self, face_value: float=100) -> None:
        self.__face_value: float = face_value
        self.__rate_tree: pd.DataFrame = None
        self.__bond_tree: pd.DataFrame = None
        self.__swap_tree: pd.DataFrame = None
        self.__cashflow_tree: pd.DataFrame = None
        self.__valuation_tree: pd.DataFrame = None

        self.__dt: float = None
        self.__T: float = None
    
    @property
    def rate_tree(self) -> pd.DataFrame:
        return self.__rate_tree

    @rate_tree.setter
    def rate_tree(self, new_rate_tree: pd.DataFrame) -> None:
        if isinstance(new_rate_tree, pd.DataFrame):
            self.__rate_tree = new_rate_tree
        else:
            raise TypeError("Can only set rate_tree attribute to pd.DataFrame type only.")
        
    @property
    def bond_tree(self) -> pd.DataFrame:
        return self.__bond_tree
    
    @bond_tree.setter
    def bond_tree(self, new_bond_tree: pd.DataFrame) -> None:
        if isinstance(new_bond_tree, pd.DataFrame):
            self.__bond_tree = new_bond_tree
        else:
            raise TypeError("Can only set bond_tree attribute to pd.DataFrame type only.")
    
    @property
    def cashflow_tree(self) -> pd.DataFrame:
        return self.__cashflow_tree

    @property
    def p_star(self):
        if (self.__rate_tree is None or self.__bond_tree is None or self.__dt is None or self.__T is None):
            raise Exception("Compute rate tree and bond tree first.")
        
        A = np.exp(self.__rate_tree.iloc[0,0] * self.__dt)

        pstar = (A * self.__bond_tree.loc[0,0] - self.__bond_tree.loc[1,self.__dt])/(self.__bond_tree.loc[0,self.__dt] - self.__bond_tree.loc[1,self.__dt])
        return pstar

    def initialize_empty_rate_tree(self, dt, T) -> None:
        self.__dt = dt
        self.__T = T
        timegrid = pd.Series((np.arange(0,round(T/dt)+1)*dt).round(6),name='time',index=pd.Index(range(round(T/dt)+1),name='state'))
        tree = pd.DataFrame(dtype=float,columns=timegrid,index=timegrid.index)
        self.__rate_tree = tree

    def compute_bond_tree(self, t0_price: float=None) -> None:
        maturity = (self.__rate_tree.columns[-1] - self.__rate_tree.columns[-2]) + self.__rate_tree.columns[-1]
        bond_tree_cols = list(self.__rate_tree.columns.copy().values)
        bond_tree_cols.append(maturity)
        print(bond_tree_cols)
        bond_tree = pd.DataFrame(dtype=float, index=pd.RangeIndex(0, len(bond_tree_cols)), columns=bond_tree_cols)
        bond_tree.index.name = self.__rate_tree.index.name
        bond_tree.columns.name = self.__rate_tree.columns.name
        
        forward_time: float = bond_tree.columns[-1]
        for time in reversed(bond_tree.columns):
            if time == maturity:
                bond_tree[time] = float(self.__face_value)
                forward_time = time
                continue
            bond_tree[time] = np.exp(-self.__rate_tree[time]*(forward_time-time)) * ((bond_tree[forward_time] + bond_tree[forward_time].shift(-1)) / 2)
            forward_time = time
        if t0_price:
            bond_tree.iloc[0,0] = t0_price
        self.__bond_tree = bond_tree

    def compute_swap_tree(self, payoff_func) -> pd.DataFrame:
        Z = np.exp(-self.__rate_tree.iloc[0,0] * self.__dt)
        swap_tree = pd.DataFrame(index=self.__rate_tree.index, columns=self.__rate_tree.columns, dtype=float)
        swap_tree[self.__dt] = payoff_func(self.__rate_tree[self.__dt])
        swap_tree.loc[0,0] = Z * np.array([self.p_star,1-self.p_star])@ swap_tree[self.__dt].values
        self.__swap_tree = swap_tree
        return swap_tree

    def construct_bond_cftree(self, T, compound, cpn, cpn_freq=2, face=100) -> pd.DataFrame:
        step = int(compound/cpn_freq)

        self.initialize_empty_rate_tree(1/compound, T)
        cftree = self.__rate_tree.copy()
        cftree.iloc[:,:] = 0
        cftree.iloc[:, -1:0:-step] = (cpn/cpn_freq) * face
        
        # final cashflow is accounted for in payoff function
        # drop final period cashflow from cashflow tree
        cftree = cftree.iloc[:-1,:-1]
        
        self.__cashflow_tree = cftree
        return cftree

    def bintree_pricing(self, payoff=None, ratetree=None, undertree=None,cftree=None, dt=None, pstars=None, timing=None, cfdelay=False,style='european',Tamerican=0):
    
        if payoff is None:
            payoff = lambda r: 0
        
        if undertree is None:
            undertree = ratetree
            
        if cftree is None:
            cftree = pd.DataFrame(0, index=undertree.index, columns=undertree.columns)
            
        if pstars is None:
            pstars = pd.Series(.5, index=undertree.columns)

        if dt is None:
            dt = undertree.columns.to_series().diff().mean()
            dt = undertree.columns[1]-undertree.columns[0]
        
        if timing == 'deferred':
            cfdelay = True
        
        if dt<.25 and cfdelay:
            display('Warning: cfdelay setting only delays by dt.')
            
        valuetree = pd.DataFrame(dtype=float, index=undertree.index, columns=undertree.columns)

        for steps_back, t in enumerate(valuetree.columns[-1::-1]):
            if steps_back==0:                           
                valuetree[t] = payoff(undertree[t])
                if cfdelay:
                    valuetree[t] *= np.exp(-ratetree[t]*dt)
            else:
                for state in valuetree[t].index[:-1]:
                    val_avg = pstars[t] * valuetree.iloc[state,-steps_back] + (1-pstars[t]) * valuetree.iloc[state+1,-steps_back]
                    
                    if cfdelay:
                        cf = cftree.loc[state,t]
                    else:                    
                        cf = cftree.iloc[state,-steps_back]
                    
                    valuetree.loc[state,t] = np.exp(-ratetree.loc[state,t]*dt) * (val_avg + cf)

                if style=='american':
                    if t>= Tamerican:
                        valuetree.loc[:,t] = np.maximum(valuetree.loc[:,t],payoff(undertree.loc[:,t]))
            
        return valuetree


    def display_rate_tree(self) -> pd.DataFrame:
        return self.__rate_tree.style.format('{:.4%}',na_rep='').format_index('{:.2f}',axis=1)
    
    def display_bond_tree(self) -> pd.DataFrame:
        return self.__bond_tree.style.format('$ {:.4}',na_rep='').format_index('{:.2f}',axis=1)

In [30]:
cap_curves_df = pd.read_excel(r'../data/cap_curves_2024-02-20.xlsx', index_col=0)
cap_curves_df.head(5)

,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
tenor,,,,,,
0.25,0.052211,0.052211,0.987115,NaN,NaN,NaN
0.50,0.051540,0.051535,0.974722,0.050860,0.166025,0.166025
0.75,0.050506,0.050490,0.963069,0.048400,0.191290,0.210648
1.00,0.049284,0.049250,0.952230,0.045531,0.216554,0.254312
1.25,0.047631,0.047565,0.942608,0.040831,0.260043,0.361247


In [31]:
cap_df = pd.read_excel(r'../data/cap_quotes_2024-02-20.xlsx')
cap_df.head(5)

,date,USCNSQ1 SMKO Curncy,USCNSQ2 SMKO Curncy,USCNSQ3 SMKO Curncy,USCNSQ4 SMKO Curncy,USCNSQ5 SMKO Curncy,USCNSQ6 SMKO Curncy,USCNSQ7 SMKO Curncy,USCNSQ8 SMKO Curncy,USCNSQ9 SMKO Curncy,USCNSQ10 SMKO Curncy
0,maturity,1.002053,2.001369,3.000684,4.0,5.002053,6.001369,7.000684,8.0,9.002053,10.001369
1,2022-03-17 00:00:00,127.300000,108.500000,109.900000,108.5,107.300000,104.300000,101.000000,97.7,95.200000,93.200000
2,2022-03-18 00:00:00,96.200000,104.900000,108.000000,108.5,108.000000,105.000000,101.800000,98.3,95.400000,93.400000
3,2022-03-21 00:00:00,96.200000,105.000000,108.100000,108.5,107.900000,105.000000,101.700000,98.2,95.400000,93.400000
4,2022-03-22 00:00:00,75.100000,108.300000,115.200000,115.3,113.600000,109.900000,106.300000,102.8,99.900000,97.500000


In [32]:
swap_vol_df = pd.read_excel(r'../data/swaption_vol_data.xlsx')
swap_vol_df

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2024-02-20,1,4,54.54,40.37,35.94,34.23,32.83,31.71,30.86,29.83,29.54


## 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

In [33]:
swaption_exp = swap_vol_df.loc[0, 'expiration']
swaption_end = swaption_exp + swap_vol_df.loc[0, 'tenor'] 
fwd_swap_rate = calc_fwdswaprate(cap_curves_df['discounts'], swaption_exp, swaption_end, freqswap=4)
fwd_swap_rate

0.03672212061985555


## 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

In [34]:
swaption_implied_df = pd.DataFrame(index=swap_vol_df.columns[-9:], columns=['IV', 'strikes', 'prices'])
swaption_implied_df['IV'] = swap_vol_df[swap_vol_df.columns[-9:]].transpose() / 100
swaption_implied_df['strikes'] = (fwd_swap_rate + swap_vol_df.columns[-9:].astype(int)/10000)
discount = cap_curves_df['discounts'].loc[1.25:5].sum()/4
# print(swaption_exp, swaption_implied_df['IV'].values, swaption_implied_df['strikes'].values, fwd_swap_rate, discount, sep='\n')
swaption_implied_df['prices'] = blacks_formula(swaption_exp, swaption_implied_df['IV'].values, swaption_implied_df['strikes'].values, fwd_swap_rate, discount)

display(swaption_implied_df)

,IV,strikes,prices
-200,0.5454,0.016722,0.072031
-100,0.4037,0.026722,0.040653
-50,0.3594,0.031722,0.027398
-25,0.3423,0.034222,0.021749
0,0.3283,0.036722,0.016874
25,0.3171,0.039222,0.012812
50,0.3086,0.041722,0.009556
100,0.2983,0.046722,0.005133
200,0.2954,0.056722,0.001473


## 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the 1x2 above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [35]:
alt_swaps_df = pd.DataFrame(columns=['exp', 'tenor', 'strike', 'prices'], index=pd.RangeIndex(0,3))
alt_swaps_df['exp'] = [0.25, 2, 1]
alt_swaps_df['tenor'] = [4, 4, 2]
alt_swaps_df['strike'] = 'ATM'

def price_swaps(exp, tenor, atm_iv, atm_strike, freq=4):
    print( exp, exp + tenor, sep='\n')
    print()
    fwd_swap_rate = calc_fwdswaprate(cap_curves_df['discounts'], exp, exp + tenor, freqswap=freq)
    discount = cap_curves_df['discounts'].loc[exp+(1/freq):exp+tenor].sum() / freq
    # print(cap_curves_df['discounts'].loc[exp+(1/freq):exp+tenor])
    # print(discount)
    print(exp, atm_iv, atm_strike, fwd_swap_rate, discount, sep='\n')
    price = blacks_formula(exp, atm_iv, atm_strike, fwd_swap_rate, discount) * 100
    return price

# alt_swaps_df['prices'] = 
price_swaps(alt_swaps_df['exp'][0], alt_swaps_df['tenor'][0], swaption_implied_df.loc[0,'IV'], swaption_implied_df.loc[0, 'strikes'])


0.25
4.25

0.25
0.3283
0.03672212061985555
0.03910405311265784
3.623807757672832


1.3951918634840146

***

# 2. Pricing w/ BDT

Use the data in `cap_curves_2024-02-20.xlsx`.

## 2.1

Calibrate the BDT Tree
* theta to fit the term structure discounts.
* sigma to fit the fwd vols from the cap data.

Report the rate tree through $T=5$. Report trees for rates compounded
* continuously
* annually

In [36]:
theta, rate_tree = estimate_theta(cap_curves_df['fwd vols'].bfill(), cap_curves_df['discounts']) 
simga = cap_curves_df['fwd vols']

In [37]:
theta.to_frame().transpose()

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
theta,NaN,-0.117846,-0.262526,-0.359865,-0.720504,-0.459257,-0.384915,-0.244081,-0.396789,-0.033089,...,0.125541,0.134603,0.153908,0.169295,0.178789,0.182346,0.179249,0.168698,0.149782,0.121431


In [64]:
# Exp
format_bintree(rate_tree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00,5.25,5.50,5.75,6.00,6.25,6.50,6.75,7.00,7.25,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
state,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.05,0.05,0.06,0.06,0.06,0.06,0.07,0.08,0.09,0.10,0.13,0.15,0.17,0.20,0.25,0.30,0.34,0.41,0.50,0.61,0.72,0.85,1.01,1.20,1.43,1.71,2.04,2.42,2.87,3.40,4.03,4.77,5.65,6.69,7.90,9.33,10.99,12.94,15.21,17.86
1,,0.05,0.05,0.05,0.05,0.05,0.06,0.07,0.07,0.09,0.11,0.13,0.14,0.17,0.21,0.25,0.29,0.34,0.42,0.51,0.61,0.72,0.86,1.02,1.21,1.45,1.73,2.05,2.43,2.88,3.41,4.04,4.79,5.66,6.69,7.90,9.31,10.96,12.89,15.13
2,,,0.04,0.04,0.04,0.04,0.05,0.06,0.06,0.07,0.09,0.10,0.12,0.14,0.17,0.20,0.24,0.28,0.34,0.42,0.49,0.58,0.69,0.82,0.98,1.17,1.40,1.66,1.97,2.33,2.76,3.27,3.88,4.59,5.42,6.40,7.54,8.88,10.44,12.26
3,,,,0.03,0.03,0.03,0.04,0.04,0.05,0.06,0.07,0.08,0.09,0.11,0.13,0.16,0.18,0.22,0.27,0.32,0.38,0.45,0.54,0.64,0.76,0.91,1.08,1.29,1.53,1.81,2.14,2.54,3.01,3.56,4.20,4.96,5.85,6.88,8.09,9.51
4,,,,,0.02,0.02,0.03,0.03,0.03,0.04,0.05,0.06,0.06,0.07,0.09,0.11,0.13,0.15,0.19,0.23,0.27,0.32,0.37,0.45,0.53,0.63,0.76,0.90,1.06,1.26,1.49,1.77,2.10,2.48,2.93,3.46,4.08,4.80,5.64,6.62
5,,,,,,0.02,0.02,0.02,0.02,0.03,0.03,0.04,0.04,0.05,0.06,0.08,0.09,0.10,0.13,0.15,0.18,0.22,0.26,0.30,0.36,0.43,0.52,0.61,0.73,0.86,1.02,1.21,1.43,1.69,2.00,2.36,2.78,3.28,3.85,4.53
6,,,,,,,0.01,0.01,0.02,0.02,0.02,0.03,0.03,0.03,0.04,0.05,0.06,0.07,0.09,0.10,0.12,0.15,0.17,0.21,0.25,0.29,0.35,0.42,0.49,0.58,0.69,0.82,0.97,1.15,1.36,1.60,1.89,2.22,2.61,3.07
7,,,,,,,,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.03,0.03,0.04,0.05,0.06,0.07,0.08,0.10,0.12,0.14,0.17,0.20,0.24,0.28,0.33,0.40,0.47,0.56,0.66,0.78,0.92,1.09,1.28,1.51,1.77,2.08
8,,,,,,,,,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.03,0.03,0.04,0.05,0.06,0.07,0.08,0.10,0.11,0.14,0.16,0.19,0.23,0.27,0.32,0.38,0.45,0.54,0.63,0.75,0.88,1.04,1.22,1.43


In [63]:
# Annual
format_bintree(np.exp(rate_tree) - 1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00,5.25,5.50,5.75,6.00,6.25,6.50,6.75,7.00,7.25,7.50,7.75,8.00,8.25,8.50,8.75,9.00,9.25,9.50,9.75
state,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.05,0.06,0.06,0.06,0.06,0.07,0.07,0.08,0.09,0.11,0.13,0.16,0.19,0.22,0.28,0.35,0.41,0.50,0.65,0.84,1.06,1.35,1.75,2.33,3.20,4.53,6.67,10.24,16.62,28.95,55.15,117.13,283.63,800.23,2699.26,11215.54,59386.87,416614.85,4048650.32,57353052.62
1,,0.05,0.05,0.05,0.05,0.06,0.06,0.07,0.08,0.09,0.11,0.14,0.16,0.18,0.23,0.29,0.34,0.41,0.53,0.67,0.84,1.06,1.35,1.77,2.37,3.26,4.61,6.77,10.36,16.81,29.32,55.93,118.90,287.11,805.28,2692.59,11051.03,57550.59,394963.26,3729856.31
2,,,0.04,0.04,0.04,0.04,0.05,0.06,0.06,0.07,0.09,0.11,0.12,0.15,0.18,0.23,0.27,0.32,0.41,0.52,0.64,0.79,1.00,1.28,1.68,2.23,3.05,4.26,6.16,9.31,14.86,25.42,47.30,97.26,225.17,599.86,1884.51,7175.78,34161.10,210600.50
3,,,,0.03,0.03,0.03,0.04,0.04,0.05,0.06,0.07,0.08,0.10,0.11,0.14,0.17,0.20,0.24,0.30,0.38,0.47,0.57,0.71,0.90,1.14,1.48,1.96,2.62,3.60,5.10,7.53,11.67,19.22,34.07,65.95,141.82,345.68,976.42,3276.45,13429.03
4,,,,,0.02,0.02,0.03,0.03,0.03,0.04,0.05,0.06,0.07,0.08,0.10,0.12,0.14,0.16,0.20,0.25,0.31,0.37,0.45,0.56,0.70,0.89,1.13,1.45,1.90,2.53,3.45,4.87,7.13,10.93,17.71,30.73,57.86,120.20,280.61,751.44
5,,,,,,0.02,0.02,0.02,0.02,0.03,0.03,0.04,0.04,0.05,0.06,0.08,0.09,0.11,0.14,0.17,0.20,0.24,0.29,0.36,0.44,0.54,0.68,0.85,1.07,1.37,1.77,2.35,3.18,4.44,6.40,9.61,15.19,25.52,46.17,91.32
6,,,,,,,0.01,0.01,0.02,0.02,0.02,0.03,0.03,0.03,0.04,0.05,0.06,0.07,0.09,0.11,0.13,0.16,0.19,0.23,0.28,0.34,0.42,0.52,0.64,0.79,1.00,1.27,1.64,2.15,2.88,3.96,5.60,8.22,12.63,20.48
7,,,,,,,,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.03,0.04,0.04,0.05,0.06,0.07,0.09,0.10,0.13,0.15,0.18,0.22,0.27,0.33,0.40,0.49,0.60,0.74,0.93,1.18,1.51,1.97,2.60,3.52,4.90,7.03
8,,,,,,,,,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.03,0.03,0.04,0.05,0.06,0.07,0.08,0.10,0.12,0.15,0.18,0.21,0.26,0.31,0.38,0.47,0.57,0.71,0.88,1.11,1.41,1.82,2.38,3.18


## 2.2

Use a tree to price a vanilla fixed-rate, 5-year bond with coupon rate equal to the forward swap rate calculated in problem `1.1.`

In [40]:
fwd_swap_rate

0.03672212061985555

In [53]:
FACE = 100
T = 5
compound = 4
cpn_freq = 2
refratetree = compound * (np.exp(rate_tree / compound)-1)

In [57]:
payoff = lambda rate: payoff_bond(rate, 1/compound, facevalue = FACE * (1 + fwd_swap_rate / cpn_freq))
cash_flow_tree = construct_bond_cftree(T, compound, fwd_swap_rate, cpn_freq=cpn_freq)
bond_tree = bintree_pricing(payoff=payoff, ratetree=rate_tree.iloc[:int(T * compound),:int(T * compound)],undertree=refratetree.iloc[:int(T * compound),:int(T * compound)], cftree=cash_flow_tree)
format_bintree(bond_tree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,98.70,97.48,94.34,92.88,89.46,87.67,83.97,82.04,78.35,76.49,73.04,71.74,69.10,68.70,67.16,68.46,69.24,73.15,77.30,86.42
1,,102.49,99.64,98.50,95.40,93.92,90.52,88.87,85.39,83.69,80.31,78.98,76.20,75.54,73.63,74.43,74.58,77.74,80.96,88.76
2,,,104.05,103.20,100.41,99.27,96.21,94.89,91.72,90.30,87.14,85.95,83.19,82.47,80.33,80.74,80.29,82.66,84.83,91.24
3,,,,106.95,104.42,103.59,100.85,99.86,97.02,95.93,93.05,92.10,89.51,88.87,86.70,86.89,85.98,87.65,88.75,93.61
4,,,,,107.49,106.90,104.42,103.73,101.17,100.39,97.81,97.13,94.74,94.27,92.18,92.31,91.13,92.34,92.62,96.10
5,,,,,,109.26,106.98,106.49,104.15,103.61,101.25,100.78,98.56,98.24,96.23,96.32,94.97,95.82,95.48,97.92
6,,,,,,,108.76,108.42,106.24,105.86,103.67,103.34,101.25,101.03,99.08,99.17,97.68,98.28,97.49,99.18
7,,,,,,,,109.75,107.68,107.41,105.33,105.11,103.10,102.96,101.05,101.12,99.53,99.97,98.86,100.03
8,,,,,,,,,108.67,108.49,106.48,106.33,104.37,104.27,102.39,102.45,100.79,101.10,99.78,100.60


In [58]:
format_bintree(bond_tree.iloc[[0],[0]])

time,0.00
state,
0,98.70


## 2.3

We will calculate the binomial tree for the 5-year swap, but here we do so by valuing the swap as...

$$\text{payer swap} = \text{floating rate note} - \text{fixed-rate bond}$$

Recall for the Floating-Rate Note:
* It has par value of 100 at each reset date.
* Every node is a reset date given the assumptions of the swap timing.

Report the tree for the 5-year swap.

In [60]:
payer_swap_tree = 100 - bond_tree
format_bintree(payer_swap_tree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,1.30,2.52,5.66,7.12,10.54,12.33,16.03,17.96,21.65,23.51,26.96,28.26,30.90,31.30,32.84,31.54,30.76,26.85,22.70,13.58
1,,-2.49,0.36,1.50,4.60,6.08,9.48,11.13,14.61,16.31,19.69,21.02,23.80,24.46,26.37,25.57,25.42,22.26,19.04,11.24
2,,,-4.05,-3.20,-0.41,0.73,3.79,5.11,8.28,9.70,12.86,14.05,16.81,17.53,19.67,19.26,19.71,17.34,15.17,8.76
3,,,,-6.95,-4.42,-3.59,-0.85,0.14,2.98,4.07,6.95,7.90,10.49,11.13,13.30,13.11,14.02,12.35,11.25,6.39
4,,,,,-7.49,-6.90,-4.42,-3.73,-1.17,-0.39,2.19,2.87,5.26,5.73,7.82,7.69,8.87,7.66,7.38,3.90
5,,,,,,-9.26,-6.98,-6.49,-4.15,-3.61,-1.25,-0.78,1.44,1.76,3.77,3.68,5.03,4.18,4.52,2.08
6,,,,,,,-8.76,-8.42,-6.24,-5.86,-3.67,-3.34,-1.25,-1.03,0.92,0.83,2.32,1.72,2.51,0.82
7,,,,,,,,-9.75,-7.68,-7.41,-5.33,-5.11,-3.10,-2.96,-1.05,-1.12,0.47,0.03,1.14,-0.03
8,,,,,,,,,-8.67,-8.49,-6.48,-6.33,-4.37,-4.27,-2.39,-2.45,-0.79,-1.10,0.22,-0.60


## 2.4



Report the binomial tree for the one-year swaption on a 4-year swap with **european** exercise.
* At expiration, the swap tree from 2.3 will have 4 years left, as desired for pricing the 1y-4y swaption.

In [59]:
payoff_swaption = lambda p: np.maximum(p, 0)

T = 1
tsteps = int(T * compound) + 1

swaption_tree = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=payer_swap_tree.iloc[:tsteps,:tsteps])
format_bintree(swaption_tree)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.72,2.92,4.80,7.46,10.54
1,,0.56,1.12,2.27,4.60
2,,,0.00,0.00,0.00
3,,,,0.00,0.00
4,,,,,0.00


## 2.5

Compare the pricing of the 1y4y swaption from Black's formula in Section 1 vs the binomial tree.

1.72 to 1.69 in section 1 which is pretty close.

## 2.6

Reprice the swaption using the BDT tree, but this time assuming it is **american**-style exercise.

In [61]:
american_style = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=payer_swap_tree.iloc[:tsteps,:tsteps], style='american')
format_bintree(american_style)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,1.93,3.35,5.66,7.46,10.54
1,,0.56,1.12,2.27,4.60
2,,,0.00,0.00,0.00
3,,,,0.00,0.00
4,,,,,0.00


***

# 3. Midcurve Swaptions

## 3.1 

Use the BDT tree from section 2 to price a **european** midcurve swaption 1y $\rightarrow$ 2y $\rightarrow$ 2y.

In [73]:
FACE = 100
T = 5
compound = 4
cpn_freq = 4

In [74]:
payoff = lambda r: payoff_bond(r, 1/compound, facevalue = FACE * (1 + fwd_swap_rate / cpn_freq))
cash_flow_tree = construct_bond_cftree(T, compound, fwd_swap_rate, cpn_freq=cpn_freq)
cash_flow_tree.iloc[:, :12] = 0
cash_flow_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053,0.918053


In [75]:
delayed_swap_tree = bintree_pricing(
    payoff=payoff, 
    ratetree=rate_tree.iloc[:int(T * compound),:int(T * compound)], 
    undertree=refratetree.iloc[:int(T * compound),:int(T * compound)], 
    cftree=cash_flow_tree
    )
delayed_swap_tree.iloc[:, 12:] = 100 - delayed_swap_tree.iloc[:, 12:]
delayed_swap_tree.iloc[:, :12] = 0
delayed_swap_tree

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.726503,32.022719,32.668818,32.260023,30.604424,27.589591,22.587467,14.357947
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.667362,25.216613,26.232674,26.322816,25.290638,23.018736,18.946467,12.035914
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.700109,18.326271,19.557472,20.044332,19.608968,18.133058,15.095819,9.585830
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.408960,11.952130,13.222253,13.932911,13.942975,13.169462,11.189356,7.233271
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.203412,6.580566,7.764096,8.541872,8.810207,8.511621,7.343087,4.765113
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.399405,2.634223,3.732634,4.543925,4.995369,5.048929,4.495009,2.965946
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.275010,-0.145824,0.886588,1.717595,2.297597,2.602309,2.490055,1.714744
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.117792,-2.059778,-1.071156,-0.224173,0.447573,0.928750,1.124328,0.870604
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.382843,-3.370279,-2.407675,-1.545303,-0.806215,-0.200288,0.208282,0.309774


In [79]:
payoff = lambda p: p

delayed_swap_tree = bintree_pricing(
    payoff=payoff, ratetree=rate_tree.iloc[:13,:13], 
    undertree=delayed_swap_tree.iloc[:13,:13]
)
format_bintree(delayed_swap_tree)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00
state,,,,,,,,,,,,,
0,-0.12,1.00,2.30,3.82,5.57,7.57,9.85,12.44,15.34,18.57,22.17,26.19,30.73
1,,-1.25,-0.28,0.85,2.18,3.73,5.53,7.62,10.03,12.79,15.95,19.55,23.67
2,,,-2.24,-1.42,-0.45,0.69,2.03,3.61,5.47,7.65,10.21,13.21,16.70
3,,,,-3.11,-2.42,-1.60,-0.64,0.50,1.85,3.45,5.36,7.65,10.41
4,,,,,-3.84,-3.27,-2.59,-1.79,-0.84,0.29,1.64,3.26,5.20
5,,,,,,-4.46,-3.99,-3.43,-2.77,-1.98,-1.05,0.06,1.40
6,,,,,,,-4.97,-4.58,-4.12,-3.58,-2.94,-2.18,-1.28
7,,,,,,,,-5.39,-5.07,-4.70,-4.25,-3.73,-3.12
8,,,,,,,,,-5.74,-5.47,-5.17,-4.81,-4.38


In [77]:
payoff_swaption = lambda p: np.maximum(p, 0)

T = 1
tsteps = int(T * compound) + 1

midcurve_swaption_tree = bintree_pricing(
    payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], 
    undertree=delayed_swap_tree.iloc[:tsteps,:tsteps]
    )
format_bintree(midcurve_swaption_tree)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.847,1.452,2.413,3.818,5.567
1,NaN,0.263,0.532,1.077,2.181
2,NaN,NaN,0.000,0.000,0.000
3,NaN,NaN,NaN,0.000,0.000
4,NaN,NaN,NaN,NaN,0.000


## 3.2

Price the **american** midcurve swaption 1y$\rightarrow$2y$\rightarrow$2y.

***

In [78]:
american_style_32 = bintree_pricing(payoff_swaption, rate_tree.iloc[:tsteps,:tsteps], undertree=delayed_swap_tree.iloc[:tsteps,:tsteps], style='american')
format_bintree(american_style_32)

time,0.00,0.25,0.50,0.75,1.00
state,,,,,
0,0.85,1.45,2.41,3.82,5.57
1,,0.26,0.53,1.08,2.18
2,,,0.00,0.00,0.00
3,,,,0.00,0.00
4,,,,,0.00
